# Before you use this template

This template is just a recommended template for project Report. It only considers the general type of research in our paper pool. Feel free to edit it to better fit your project. You will iteratively update the same notebook submission for your draft and the final submission. Please check the project rubriks to get a sense of what is expected in the template.

---

# FAQ and Attentions
* Copy and move this template to your Google Drive. Name your notebook by your team ID (upper-left corner). Don't eidt this original file.
* This template covers most questions we want to ask about your reproduction experiment. You don't need to exactly follow the template, however, you should address the questions. Please feel free to customize your report accordingly.
* any report must have run-able codes and necessary annotations (in text and code comments).
* The notebook is like a demo and only uses small-size data (a subset of original data or processed data), the entire runtime of the notebook including data reading, data process, model training, printing, figure plotting, etc,
must be within 8 min, otherwise, you may get penalty on the grade.
  * If the raw dataset is too large to be loaded  you can select a subset of data and pre-process the data, then, upload the subset or processed data to Google Drive and load them in this notebook.
  * If the whole training is too long to run, you can only set the number of training epoch to a small number, e.g., 3, just show that the training is runable.
  * For results model validation, you can train the model outside this notebook in advance, then, load pretrained model and use it for validation (display the figures, print the metrics).
* The post-process is important! For post-process of the results,please use plots/figures. The code to summarize results and plot figures may be tedious, however, it won't be waste of time since these figures can be used for presentation. While plotting in code, the figures should have titles or captions if necessary (e.g., title your figure with "Figure 1. xxxx")
* There is not page limit to your notebook report, you can also use separate notebooks for the report, just make sure your grader can access and run/test them.
* If you use outside resources, please refer them (in any formats). Include the links to the resources if necessary.

# Mount Notebook to Google Drive
Upload the data, pretrianed model, figures, etc to your Google Drive, then mount this notebook to Google Drive. After that, you can access the resources freely.

Instruction: https://colab.research.google.com/notebooks/io.ipynb

Example: https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q

Video: https://www.youtube.com/watch?v=zc8g8lGcwQU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Introduction

*Background*

Surgery is often associated with fluctuations in blood pressure.(Salmasi V, Maheshwari K, Yang D, Mascha EJ, Singh A, Sessler DI, et al. Relationship between intraoperative hypotension, defined by either reduction from baseline or absolute thresholds, and acute kidney and myocardial injury after noncardiac surgery: a retrospective cohort analysis. Anesthesiology. 2017;126(1):47–65. pmid:27792044) Low blood pressure (‘intraoperative hypotension’) might occur due to medications administered (such as sedation) or blood loss. Hypotension is potentially dangerous as it might lead to reduced blood flow to vital organs such as the heart or the brain. Therefore, careful monitoring of the intraoperative blood pressure is performed in order to treat hypotensive events (usually defined as a mean arterial blood pressure [MAP] <65 mmHg) if they occur (commonly with fluids or medications). Continuously measured parameters such as the MAP, the patient’s ECG, or EEG might allow for earlier prediction of subsequent hypotensive events. This could, in turn, enable a more timely intervention and potentially even prevention of hypotensive events.


*Paper explanation*

Specific approach: This paper uses a public data repository of vital signs taken during surgery in 10 operating rooms at Seoul National University Hospital between 01/06/2005 and 03/01/2024. The final analysis included 14,140 patients undergoing non-cardiac surgery. Arterial blood pressure (ABP), Electrocardiogram (ECG), and Electroencephalogram (EEG) waveforms obtained during surgery were used to predict hypotensive events. Specifically, 1-min intervals of the waveforms were sampled 3, 5, 10, and 15 min before a hypotensive event (defined as a MAP<65 mmHg ≥1 min) and compared to waveforms prior to ‘non-events’ (samples in the middle of a 30 min window of a MAP ≥75 mmHg). Unreliable cases were removed using the J signal quality index (Li Q., Mark R.G. & Clifford G.D. Artificial arterial blood pressure artifact models and an evaluation of a robust blood pressure and heart rate estimator. BioMed Eng OnLine. 2009; 8(13). pmid:19586547).
Following data preprocessing, the authors trained a ResNet CNN for each waveform. The outputs are subsequently concatenated and passed through a classifier to predict hypotensive events.


# Scope of Reproducibility:

*Hypothesis:*

Is it possible to predict intraoperative hypotensive events using a deep-learning based analysis of MAP, ECG, and EEG waveforms?

*Experimental setup:*

Individual analysis of the predictive performance of the ABP, ECG, and EEG data compared to the model concatenating the results in order to understand the benefit vs the cost associated with the additional computation. Additional ablations will include studying the effect of the optimizer (Adam was used for the final model) and the learning rate (0.0001 in the paper).


# Methodology


Link to github repo: https://github.com/sebbeyer/DLH_project_168.git

## **Install and load packages**

In [ ]:
!pip install vitaldb

In [ ]:
# import  packages you need
import os
import sys
import glob
import math
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import vitaldb
import scipy.signal
import scipy.io.wavfile

##  **Load Data**

*Source of the data:*

The dataset used is open access: https://osf.io/dtc45/. It can be obtained after signing the Data Use Agreement (https://vitaldb.net/docs/?documentId=1OyhiDYbN-VJ6TOme-Fkj4wbqJkVT3UazELcbCXcHmiY)

To run the notebook:
1) Download the .vital files after signing the Data Use Agreement (each .vital file corresponds to the tracings of one patient during surgery)
2) update 'raw_data_dir'

Because of limited computing resources, this analysis will be limited to the first 2,000 patients included in the dataset.

In [ ]:
# show available tracks (all vital sign tracings obtained during surgery)
tracks = vitaldb.vital_trks('/content/drive/MyDrive/VitalDB/vital_files_1_250/0001.vital')
tracks

In [ ]:
def load_raw_data(raw_data_dir):

  # load data to listsy with events and non-events
  # generate separate lists for 3, 5, 10, and 15 min preceding events

  ecg_3min = []
  art_3min = []
  eeg_3min = []
  y_3min = []

  ecg_5min = []
  art_5min = []
  eeg_5min = []
  y_5min = []

  ecg_10min = []
  art_10min = []
  eeg_10min = []
  y_10min = []

  ecg_15min = []
  art_15min = []
  eeg_15min = []
  y_15min = []

  # tracks to load
  tracks = ['SNUADC/ECG_II', 'SNUADC/ART', 'BIS/EEG1_WAV'] # the paper does not specify which ECG lead they used for the analysis -> Here, we are using lead II
  # the paper also does not specify which EEG waveform is being used -> Here, we are using waveform 1

  # iterate over all files
  for file in os.listdir(raw_data_dir):

    #  extract ecg, blood pressure (art) and eeg data as numpy array
    vital_object = vitaldb.VitalFile(file, tracks)
    ecg_np = vital_object.to_numpy('SNUADC/ECG_II', 1/500)
    art_np = vital_object.to_numpy('SNUADC/ART', 1/500)
    eeg_np = vital_object.to_numpy('BIS/EEG1_WAV', 1/128)

    # calculate mean arterial blood pressure (MAP) for 1 minute intervals
    map = []

    for i in range(0, art_np.shape[0], 30000): # sampling of 500Hz -> 30,000 datapoints / min
      map.append(np.mean(art_np[i:i+30000]))

    map = np.asarray(map)

    # Quality check - exclude implausible values
    map = np.where(map<20, np.nan, map)
    map = np.where(map>200, np.nan, map)

    # find index of events
    # hypotensive events: MAP<65 mmHg

    # index of hypotensive events mandating >20 min between each
    # hypotensive event (the paper does not specify whether this is
    # 20 min after the hypotension has completely resolved or 20 min
    # after the onset of hypotension)
    # -> here, I will require >20 min following the resolution of
    # hypotension

    # even more importantly, the authors did not specify how the following
    # scenario should be handled: a second hypotensive event w/in 20 min
    # of a prior hypotensive event with a third hypotensive event w/in 20 min
    # of the second hypotensive event but >20 min after the first hypotensive
    # event: should the third hypotensive event be considered an event and
    # included in the analysis???
    # -> here, I excluded even the third event as there are <20 min between
    # consecutive hypotensive events

    # the authors also didn't specify how they dealt with MAP>75 mmHg
    # segemnts lasting >30 (i.e. when during those longer interavls they
    # sampled controls)
    # -> here, I will use the initial 30 min of such intervals

    # 'events' array as indicator array:
      # '0' : MAP 65 - 75 mmHg
      # '1': MAP <65 mmHg with >20 min since the last hypotensive event
      # '-1': MAP <65 mmHg with <= 20 min since last hypotensive event
      # '2': MAP > 75 mmHg for 30 min (initial 30 min if MAP>75 for >30 min)

    events = (map<65)*1 # MAP<65
    events = np.where(np.isnan(map), np.nan, events) # keep nan as nan intervals

    prec_20_min = np.asarray([np.nansum(events[max(i[0]-20, 0):i[0]]) for i in enumerate(events)]) # check for hypotensive events during 20 min interval preceding that timestamp
    prec_20_min[prec_20_min >0] = 2

    events = events - prec_20_min
    events[events < -1] = 0

    map_75 = (map>75)*2 # MAP>75
    map_75 = np.where(np.isnan(map), np.nan, map_75) # keep nan as nan intervals

    prec_30_min = np.asarray([np.nansum(map_75[max(i[0]-30, 0):i[0]]) for i in enumerate(map_75)]) # check for MAP>75 lasting >= 30 min
    prec_30_min = (prec_30_min == 60).astype(int)

    prec_30_min_2 = np.asarray([np.nansum(prec_30_min[max(i[0]-30, 0):i[0]]) for i in enumerate(prec_30_min)]) # identify initial 30 min w/ MAP >75 if longer time interval >75 mmHg
    prec_30_min_2[prec_30_min_2 >0] = 2

    map_75 = (map_75 * prec_30_min) - prec_30_min_2
    map_75[map_75 == -2] = 0

    events = events + map_75

    # append lists

    for i, j in enumerate(events):
      if j == 1:
        if (i >2 and np.isnan(ecg_np[(i-3)*30000:(i-2)*30000]).any() == False and
        np.isnan(art_np[(i-3)*30000:(i-2)*30000]).any() == False and
        np.isnan(eeg_np[(i-3)*7680:(i-2)*7680]).any() == False and
        np.isnan(map[i-3]) == False):

          ecg_3min.append(ecg_np[(i-3)*30000:(i-2)*30000])
          art_3min.append(art_np[(i-3)*30000:(i-2)*30000])
          eeg_3min.append(eeg_np[(i-3)*7680:(i-2)*7680])
          y_3min.append(1)

        if (i >4 and np.isnan(ecg_np[(i-5)*30000:(i-4)*30000]).any() == False and
        np.isnan(art_np[(i-5)*30000:(i-4)*30000]).any() == False and
        np.isnan(eeg_np[(i-5)*7680:(i-4)*7680]).any() == False and
        np.isnan(map[i-5]) == False):

          ecg_5min.append(ecg_np[(i-5)*30000:(i-4)*30000])
          art_5min.append(art_np[(i-5)*30000:(i-4)*30000])
          eeg_5min.append(eeg_np[(i-5)*7680:(i-4)*7680])
          y_5min.append(1)

        if (i >9 and np.isnan(ecg_np[(i-10)*30000:(i-9)*30000]).any() == False and
        np.isnan(art_np[(i-10)*30000:(i-9)*30000]).any() == False and
        np.isnan(eeg_np[(i-10)*7680:(i-9)*7680]).any() == False and
        np.isnan(map[i-10]) == False):

          ecg_10min.append(ecg_np[(i-10)*30000:(i-9)*30000])
          art_10min.append(art_np[(i-10)*30000:(i-9)*30000])
          eeg_10min.append(eeg_np[(i-10)*7680:(i-9)*7680])
          y_10min.append(1)

        if (i >14 and np.isnan(ecg_np[(i-15)*30000:(i-14)*30000]).any() == False and
        np.isnan(art_np[(i-15)*30000:(i-14)*30000]).any() == False and
        np.isnan(eeg_np[(i-15)*7680:(i-14)*7680]).any() == False and
        np.isnan(map[i-15]) == False):

          ecg_15min.append(ecg_np[(i-15)*30000:(i-14)*30000])
          art_15min.append(art_np[(i-15)*30000:(i-14)*30000])
          eeg_15min.append(eeg_np[(i-15)*7680:(i-14)*7680])
          y_15min.append(1)

      if (j == 2 and np.isnan(ecg_np[(i-15)*30000:(i-14)*30000]).any() == False and
      np.isnan(art_np[(i-15)*30000:(i-14)*30000]).any() == False and
      np.isnan(eeg_np[(i-15)*7680:(i-14)*7680]).any() == False and
      np.isnan(map[i-15]) == False):

        ecg_3min.append(ecg_np[(i-15)*30000:(i-14)*30000])
        ecg_5min.append(ecg_np[(i-15)*30000:(i-14)*30000])
        ecg_10min.append(ecg_np[(i-15)*30000:(i-14)*30000])
        ecg_15min.append(ecg_np[(i-15)*30000:(i-14)*30000])
        art_3min.append(art_np[(i-15)*30000:(i-14)*30000])
        art_5min.append(art_np[(i-15)*30000:(i-14)*30000])
        art_10min.append(art_np[(i-15)*30000:(i-14)*30000])
        art_15min.append(art_np[(i-15)*30000:(i-14)*30000])
        eeg_3min.append(eeg_np[(i-15)*7680:(i-14)*7680])
        eeg_5min.append(eeg_np[(i-15)*7680:(i-14)*7680])
        eeg_10min.append(eeg_np[(i-15)*7680:(i-14)*7680])
        eeg_15min.append(eeg_np[(i-15)*7680:(i-14)*7680])
        y_3min.append(0)
        y_5min.append(0)
        y_10min.append(0)
        y_15min.append(0)

  return (ecg_3min, art_3min, eeg_3min, y_3min, ecg_5min, art_5min, eeg_5min, y_5min,
         ecg_10min, art_10min, eeg_10min, y_10min, ecg_15min, art_15min, eeg_15min, y_15min)


In [ ]:
raw_data_dir = ['/content/drive/MyDrive/VitalDB/vital_files_1_250',
                '/content/drive/MyDrive/VitalDB/vital_files_251_500',
                '/content/drive/MyDrive/VitalDB/vital_files_501_750',
                '/content/drive/MyDrive/VitalDB/vital_files_751_1000',
                '/content/drive/MyDrive/VitalDB/vital_files_1001_1250',
                '/content/drive/MyDrive/VitalDB/vital_files_1251_1500',
                '/content/drive/MyDrive/VitalDB/vital_files_1501_1750',
                '/content/drive/MyDrive/VitalDB/vital_files_1751_2000']

ecg_3min = []
art_3min = []
eeg_3min = []
y_3min = []

ecg_5min = []
art_5min = []
eeg_5min = []
y_5min = []

ecg_10min = []
art_10min = []
eeg_10min = []
y_10min = []

ecg_15min = []
art_15min = []
eeg_15min = []
y_15min = []

for i in raw_data_dir:

  os.chdir(i)

  j = load_raw_data(i)
  ecg_3min.extend(j[0])
  art_3min.extend(j[1])
  eeg_3min.extend(j[2])
  y_3min.extend(j[3])

  ecg_5min.extend(j[4])
  art_5min.extend(j[5])
  eeg_5min.extend(j[6])
  y_5min.extend(j[7])

  ecg_10min.extend(j[8])
  art_10min.extend(j[9])
  eeg_10min.extend(j[10])
  y_10min.extend(j[11])

  ecg_15min.extend(j[12])
  art_15min.extend(j[13])
  eeg_15min.extend(j[14])
  y_15min.extend(j[15])

In [ ]:
# convert to np array and save copy to google drive

ecg_3min = np.asarray(ecg_3min)
art_3min = np.asarray(art_3min)
eeg_3min = np.asarray(eeg_3min)
y_3min = np.asarray(y_3min)

ecg_5min = np.asarray(ecg_5min)
art_5min = np.asarray(art_5min)
eeg_5min = np.asarray(eeg_5min)
y_5min = np.asarray(y_5min)

ecg_10min = np.asarray(ecg_10min)
art_10min = np.asarray(art_10min)
eeg_10min = np.asarray(eeg_10min)
y_10min = np.asarray(y_10min)

ecg_15min = np.asarray(ecg_15min)
art_15min = np.asarray(art_15min)
eeg_15min = np.asarray(eeg_15min)
y_15min = np.asarray(y_15min)

os.chdir('/content/drive/MyDrive/VitalDB')

np.save('/content/drive/MyDrive/VitalDB/ecg_3min2.npy', ecg_3min)
np.save('/content/drive/MyDrive/VitalDB/art_3min2.npy', art_3min)
np.save('/content/drive/MyDrive/VitalDB/eeg_3min2.npy', eeg_3min)
np.save('/content/drive/MyDrive/VitalDB/y_3min2.npy', y_3min)

np.save('/content/drive/MyDrive/VitalDB/ecg_5min2.npy', ecg_5min)
np.save('/content/drive/MyDrive/VitalDB/art_5min2.npy', art_5min)
np.save('/content/drive/MyDrive/VitalDB/eeg_5min2.npy', eeg_5min)
np.save('/content/drive/MyDrive/VitalDB/y_5min2.npy', y_5min)

np.save('/content/drive/MyDrive/VitalDB/ecg_10min2.npy', ecg_10min)
np.save('/content/drive/MyDrive/VitalDB/art_10min2.npy', art_10min)
np.save('/content/drive/MyDrive/VitalDB/eeg_10min2.npy', eeg_10min)
np.save('/content/drive/MyDrive/VitalDB/y_10min2.npy', y_10min)

np.save('/content/drive/MyDrive/VitalDB/ecg_15min2.npy', ecg_15min)
np.save('/content/drive/MyDrive/VitalDB/art_15min2.npy', art_15min)
np.save('/content/drive/MyDrive/VitalDB/eeg_15min2.npy', eeg_15min)
np.save('/content/drive/MyDrive/VitalDB/y_15min2.npy', y_15min)

In [ ]:
# load copies
ecg_3min = np.load('/content/drive/MyDrive/VitalDB/ecg_3min.npy')
art_3min = np.load('/content/drive/MyDrive/VitalDB/art_3min.npy')
eeg_3min = np.load('/content/drive/MyDrive/VitalDB/eeg_3min.npy')
y_3min = np.load('/content/drive/MyDrive/VitalDB/y_3min.npy')

ecg_5min = np.load('/content/drive/MyDrive/VitalDB/ecg_5min.npy')
art_5min = np.load('/content/drive/MyDrive/VitalDB/art_5min.npy')
eeg_5min = np.load('/content/drive/MyDrive/VitalDB/eeg_5min.npy')
y_5min = np.load('/content/drive/MyDrive/VitalDB/y_5min.npy')

ecg_10min = np.load('/content/drive/MyDrive/VitalDB/ecg_10min.npy')
art_10min = np.load('/content/drive/MyDrive/VitalDB/art_10min.npy')
eeg_10min = np.load('/content/drive/MyDrive/VitalDB/eeg_10min.npy')
y_10min = np.load('/content/drive/MyDrive/VitalDB/y_10min.npy')

ecg_15min = np.load('/content/drive/MyDrive/VitalDB/ecg_15min.npy')
art_15min = np.load('/content/drive/MyDrive/VitalDB/art_15min.npy')
eeg_15min = np.load('/content/drive/MyDrive/VitalDB/eeg_15min.npy')
y_15min = np.load('/content/drive/MyDrive/VitalDB/y_15min.npy')

## **Preprocessing**

In [ ]:
# Apply frequency filter
# While the frequencies are provided in the paper, additional technical
# details such as the type of filter or the filter settings are not mentioned
# -> here, I am using a 4-th order Butterworth filter

def bandpass(data, edges, sampling_rate, poles: int = 4):
    sos = scipy.signal.butter(poles, edges, 'bandpass', fs=sampling_rate, output='sos')
    filtered_data = scipy.signal.sosfiltfilt(sos, data, axis=1)
    return filtered_data

In [ ]:
# Normalizing ECGs using Z scores
# It is not clear from the paper whether Z score are calculated for each sample
# or for the entire dataset

def normalize(data):

  mean_data = np.mean(data)
  sd_data = np.std(data)
  normalized_data = (data - mean_data) / sd_data
  return normalized_data

In [ ]:
# process raw data
sampling_rate_ecg = 500
fmin_ecg = 1
fmax_ecg = 40

sampling_rate_eeg = 128
fmin_eeg = 0.5
fmax_eeg = 50

def process_data(ecg_3min, ecg_5min, ecg_10min, ecg_15min, eeg_3min, eeg_5min, eeg_10min, eeg_15min,
                 sampling_rate_ecg, fmin_ecg, fmax_ecg, sampling_rate_eeg, fmin_eeg, fmax_eeg):

  # bandpass filter
  ecg_3min_filtered = bandpass(ecg_3min, [fmin_ecg, fmax_ecg], sampling_rate_ecg)
  ecg_5min_filtered = bandpass(ecg_5min, [fmin_ecg, fmax_ecg], sampling_rate_ecg)
  ecg_10min_filtered = bandpass(ecg_10min, [fmin_ecg, fmax_ecg], sampling_rate_ecg)
  ecg_15min_filtered = bandpass(ecg_15min, [fmin_ecg, fmax_ecg], sampling_rate_ecg)

  eeg_3min_filtered = bandpass(eeg_3min, [fmin_eeg, fmax_eeg], sampling_rate_eeg)
  eeg_5min_filtered = bandpass(eeg_5min, [fmin_eeg, fmax_eeg], sampling_rate_eeg)
  eeg_10min_filtered = bandpass(eeg_10min, [fmin_eeg, fmax_eeg], sampling_rate_eeg)
  eeg_15min_filtered = bandpass(eeg_15min, [fmin_eeg, fmax_eeg], sampling_rate_eeg)

  # normalizing ECG using Z score
  ecg_3min_normalized = normalize(ecg_3min_filtered)
  ecg_5min_normalized = normalize(ecg_5min_filtered)
  ecg_10min_normalized = normalize(ecg_10min_filtered)
  ecg_15min_normalized = normalize(ecg_15min_filtered)

  return ecg_3min_normalized, ecg_5min_normalized, ecg_10min_normalized, ecg_15min_normalized, \
        eeg_3min_filtered, eeg_5min_filtered, eeg_10min_filtered, eeg_15min_filtered

ecg_3min_normalized, ecg_5min_normalized, ecg_10min_normalized, ecg_15min_normalized, \
eeg_3min_filtered, eeg_5min_filtered, eeg_10min_filtered, eeg_15min_filtered = \
process_data(ecg_3min, ecg_5min, ecg_10min, ecg_15min, eeg_3min, eeg_5min, eeg_10min, eeg_15min,
                 sampling_rate_ecg, fmin_ecg, fmax_ecg, sampling_rate_eeg, fmin_eeg, fmax_eeg)

## **Statistics and Sample Tracings**

In [ ]:
# Statistics

# calculate statistics
def calculate_stats(y):

  n_samples_3min = len(y)
  cases_3min = np.sum(y)
  controls_3min = len(y) - np.sum(y)

  return n_samples_3min, cases_3min, controls_3min

n_samples_3min, cases_3min, controls_3min = calculate_stats(y_3min)

print(f'total number of samples with at least 3 minutes of data prior to event: {n_samples_3min}')
print(f'total number of cases with at least 3 minutes of data prior to event: {cases_3min}')
print(f'total number of controls: {controls_3min}')

In [ ]:
# Plot sample waveforms

ecg = ecg_3min_normalized[0,:]
art = art_3min[0,:]
eeg = eeg_3min_filtered[0,:]

plt.figure(figsize=(20,10))
plt.subplot(311)
plt.plot(ecg[0:1000], color='g')
plt.subplot(312)
plt.plot(art[0:1000], color='r')
plt.subplot(313)
plt.plot(eeg[0:1000], color='b')
plt.show()

## **Train / Test Split and Dataset / Dataloader**

In [ ]:
# Define training and test samples (80:20 split)
# Since each patient might contribute multiple samples, shuffling will not be used at this point

def split_train_test(ecg, art, eeg, y, ratio):

  split = int(ratio * len(y))

  ecg_train = ecg[:split]
  ecg_test = ecg[split:]
  art_train = art[:split]
  art_test = art[split:]
  eeg_train = eeg[:split]
  eeg_test = eeg[split:]
  y_train = y[:split]
  y_test = y[split:]

  return ecg_train, ecg_test, art_train, art_test, eeg_train, eeg_test, y_train, y_test

ecg_3min_train, ecg_3min_test, art_3min_train, art_3min_test, eeg_3min_train, eeg_3min_test, y_3min_train, y_3min_test = \
split_train_test(ecg_3min_normalized, art_3min, eeg_3min_filtered, y_3min, 0.8)

ecg_5min_train, ecg_5min_test, art_5min_train, art_5min_test, eeg_5min_train, eeg_5min_test, y_5min_train, y_5min_test = \
split_train_test(ecg_5min_normalized, art_5min, eeg_5min_filtered, y_5min, 0.8)

ecg_10min_train, ecg_10min_test, art_10min_train, art_10min_test, eeg_10min_train, eeg_10min_test, y_10min_train, y_10min_test = \
split_train_test(ecg_10min_normalized, art_10min, eeg_10min_filtered, y_10min, 0.8)

ecg_15min_train, ecg_15min_test, art_15min_train, art_15min_test, eeg_15min_train, eeg_15min_test, y_15min_train, y_15min_test = \
split_train_test(ecg_15min_normalized, art_15min, eeg_15min_filtered, y_15min, 0.8)

In [ ]:
# Define Custom Dataset class
from torch.utils.data import Dataset

class HypoDataset(Dataset):

    def __init__(self, ecg, art, eeg, y):

        super().__init__()
        self.y = y
        self.ecg = ecg
        self.art = art
        self.eeg = eeg

    def __len__(self):

        return len(self.y)

    def __getitem__(self, i):

        return ((self.ecg[i], self.art[i], self.eeg[i]), self.y[i])

In [ ]:
# Define function to load dataset
from torch.utils.data import DataLoader

def load_data(dataset, batch_size=32):
    """
    Return a DataLoader instance basing on a Dataset instance, with batch_size specified.
    """
    def my_collate(batch):

        # your code here
        x, y = zip(*batch)
        ecg, art, eeg = zip(*x)

        Y = torch.tensor(y, dtype=torch.float)

        ECG = torch.tensor(ecg, dtype=torch.float).transpose(1,2)
        ART = torch.tensor(art, dtype=torch.float).transpose(1,2)
        EEG = torch.tensor(eeg, dtype=torch.float).transpose(1,2)

        return (ECG, ART, EEG), Y

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=my_collate)

In [ ]:
train_loader_3min = load_data(HypoDataset(ecg_3min_train, art_3min_train, eeg_3min_train, y_3min_train))
test_loader_3min = load_data(HypoDataset(ecg_3min_test, art_3min_test, eeg_3min_test, y_3min_test))

train_loader_5min = load_data(HypoDataset(ecg_5min_train, art_5min_train, eeg_5min_train, y_5min_train))
test_loader_5min = load_data(HypoDataset(ecg_5min_test, art_5min_test, eeg_5min_test, y_5min_test))

train_loader_10min = load_data(HypoDataset(ecg_10min_train, art_10min_train, eeg_10min_train, y_10min_train))
test_loader_10min = load_data(HypoDataset(ecg_10min_test, art_10min_test, eeg_10min_test, y_10min_test))

train_loader_15min = load_data(HypoDataset(ecg_15min_train, art_15min_train, eeg_15min_train, y_15min_train))
test_loader_15min = load_data(HypoDataset(ecg_15min_test, art_15min_test, eeg_15min_test, y_15min_test))

## **Model Architecture**

In [ ]:
import torch.nn.functional as F

# the model architecture and details are poorly described and
# discrepant. quite frankly, this makes me question the choice of
# this paper as an option for a final project in this class:

# 1)
# the text mentions an additional encoder block (conv + dropout)
# (before the data are passed through the residual blocks), which
# is not shown in Fig 2 or Suppl. Table 1. Even more concerning, in the
# text it says that the encoder blocks consist of a conv layer and a
# max pooling layer whose technical specifications aren't mentioned
# at all

# 2)
# at least some of the conv layers have to use padding since residual
# connections are being used. However, padding is not mentioned at all

# 3)
# Supple Table 1 (detailing the hyperparameter settings) is not
# consistent with the description of the model in the text of Fig 2:
# the output size in Suppl Table 1 implies pooling layers are being
# used between every other residual layer, but this is inconsistent
# with the text or figure

# 4)
# According to Suppl. Table 1, channels increases w/ subsequent residual blocks,
# but according to Fig 2 each layer has to conv layers - ???which layer
# increases the channels???

# 5)
# According to Suppl. Table 1, kernel sizes change with subsequent
# residual blocks - this is contradictory to what is mentioned in the text

# 6)
# What kind of activation function do the linear layers have? Relu???

# 7)
# Some residual blocks increase the number of channels -> a residual
# connection is not possible here (unless the number of channels of the
# input is also being adjusted for the skip connection, which is
# also not mentioned at all in the text)

# 8)
# The output size numbers provided in Suppl Table 1 don't add up:
# Max pooling w/ (2,2) applied to 1875 results in 937, not 938
# Max pooling w/ (2,2) applied to 937 results in 468 (and even
# max pooling applied to 938 does not result in 496)

# 9)
# it is unclear how weights were initialized

# 10)
# The authors do not mention the software / package they used for the analysis!

class my_model(nn.Module):
  # use this class to define your model
  def __init__(self):

    super().__init__()

    self.encoder_ecg = nn.Sequential(
                        nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3,stride=1, padding='same')
                        )

    self.layer1_ecg = nn.Sequential(
                        nn.BatchNorm1d(1),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(1,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.maxpool_ecg_layer1 = nn.MaxPool1d(2, 2)

    self.layer2_ecg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.layer3_ecg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.maxpool_ecg_layer3 = nn.MaxPool1d(2, 2)

    self.layer4_ecg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.layer5_ecg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.maxpool_ecg_layer5 = nn.MaxPool1d(2, 2)

    self.layer6_ecg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,4,15,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,15,stride=1, padding='same')
                        )

    self.layer7_ecg = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,4,7,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,7,stride=1, padding='same')
                        )

    self.maxpool_ecg_layer7 = nn.MaxPool1d(2, 2)

    self.layer8_ecg = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,4,7,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,7,stride=1, padding='same')
                        )

    self.layer9_ecg = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,4,7,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,7,stride=1, padding='same')
                        )

    self.maxpool_ecg_layer9 = nn.MaxPool1d(2, 2)

    self.layer10_ecg = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,6,7,stride=1, padding='same'),
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Conv1d(6,6,7,stride=1, padding='same')
                        )

    self.layer11_ecg = nn.Sequential(
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(6,6,7,stride=1, padding='same'),
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Conv1d(6,6,7,stride=1, padding='same')
                        )

    self.maxpool_ecg_layer11 = nn.MaxPool1d(2, 2)

    self.layer12_ecg = nn.Sequential(
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(6,6,7,stride=1, padding='same'),
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Conv1d(6,6,7,stride=1, padding='same')
                        )

    self.linear_ecg = nn.Linear(468*6, 32)

    self.encoder_art = nn.Sequential(
                        nn.Conv1d(1,1,3,stride=1, padding='same')
                        )

    self.layer1_art = nn.Sequential(
                        nn.BatchNorm1d(1),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(1,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.maxpool_art_layer1 = nn.MaxPool1d(2, 2)

    self.layer2_art = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.layer3_art = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.maxpool_art_layer3 = nn.MaxPool1d(2, 2)

    self.layer4_art = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.layer5_art = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,15,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,15,stride=1, padding='same')
                        )

    self.maxpool_art_layer5 = nn.MaxPool1d(2, 2)

    self.layer6_art = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,4,15,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,15,stride=1, padding='same')
                        )

    self.layer7_art = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,4,7,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,7,stride=1, padding='same')
                        )

    self.maxpool_art_layer7 = nn.MaxPool1d(2, 2)

    self.layer8_art = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,4,7,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,7,stride=1, padding='same')
                        )

    self.layer9_art = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,4,7,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,7,stride=1, padding='same')
                        )

    self.maxpool_art_layer9 = nn.MaxPool1d(2, 2)

    self.layer10_art = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,6,7,stride=1, padding='same'),
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Conv1d(6,6,7,stride=1, padding='same')
                        )

    self.layer11_art = nn.Sequential(
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(6,6,7,stride=1, padding='same'),
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Conv1d(6,6,7,stride=1, padding='same')
                        )

    self.maxpool_art_layer11 = nn.MaxPool1d(2, 2)

    self.layer12_art = nn.Sequential(
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(6,6,7,stride=1, padding='same'),
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Conv1d(6,6,7,stride=1, padding='same')
                        )

    self.linear_art = nn.Linear(468*6, 32)

    self.encoder_eeg = nn.Sequential(
                        nn.Conv1d(1,1,3,stride=1, padding='same')
                        )

    self.layer1_eeg = nn.Sequential(
                        nn.BatchNorm1d(1),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(1,2,7,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,7,stride=1, padding='same')
                        )

    self.maxpool_eeg_layer1 = nn.MaxPool1d(2, 2)

    self.layer2_eeg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,7,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,7,stride=1, padding='same')
                        )

    self.layer3_eeg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,7,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,7,stride=1, padding='same')
                        )

    self.maxpool_eeg_layer3 = nn.MaxPool1d(2, 2)

    self.layer4_eeg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,7,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,7,stride=1, padding='same')
                        )

    self.layer5_eeg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,2,7,stride=1, padding='same'),
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Conv1d(2,2,7,stride=1, padding='same')
                        )

    self.maxpool_eeg_layer5 = nn.MaxPool1d(2, 2)

    self.layer6_eeg = nn.Sequential(
                        nn.BatchNorm1d(2),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(2,4,7,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,7,stride=1, padding='same')
                        )

    self.layer7_eeg = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,4,3,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,3,stride=1, padding='same')
                        )

    self.maxpool_eeg_layer7 = nn.MaxPool1d(2, 2)

    self.layer8_eeg = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,4,3,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,3,stride=1, padding='same')
                        )

    self.layer9_eeg = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,4,3,stride=1, padding='same'),
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Conv1d(4,4,3,stride=1, padding='same')
                        )

    self.maxpool_eeg_layer9 = nn.MaxPool1d(2, 2)

    self.layer10_eeg = nn.Sequential(
                        nn.BatchNorm1d(4),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(4,6,3,stride=1, padding='same'),
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Conv1d(6,6,3,stride=1, padding='same')
                        )

    self.layer11_eeg = nn.Sequential(
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(6,6,3,stride=1, padding='same'),
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Conv1d(6,6,3,stride=1, padding='same')
                        )

    self.maxpool_eeg_layer11 = nn.MaxPool1d(2, 2)

    self.layer12_eeg = nn.Sequential(
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Dropout(),
                        nn.Conv1d(6,6,3,stride=1, padding='same'),
                        nn.BatchNorm1d(6),
                        nn.ReLU(),
                        nn.Conv1d(6,6,3,stride=1, padding='same')
                        )

    self.linear_eeg = nn.Linear(120*6, 32)

    self.linear_combined1 = nn.Linear(96, 16)
    self.linear_combined2 = nn.Linear(16, 1)

  def forward(self, ecg, art, eeg):

    ecg = self.encoder_ecg(ecg)
    tmp = self.layer1_ecg(ecg)
    ecg = tmp + ecg
    ecg = self.maxpool_ecg_layer1(ecg)
    tmp = self.layer2_ecg(ecg)
    ecg = tmp + ecg
    tmp = self.layer3_ecg(ecg)
    ecg = tmp + ecg
    ecg = self.maxpool_ecg_layer3(ecg)
    tmp = self.layer4_ecg(ecg)
    ecg = tmp + ecg
    tmp = self.layer5_ecg(ecg)
    ecg = tmp + ecg
    ecg = self.maxpool_ecg_layer5(ecg)
    ecg = self.layer6_ecg(ecg)
    tmp = self.layer7_ecg(ecg)
    ecg = tmp + ecg
    ecg = self.maxpool_ecg_layer7(ecg)
    tmp = self.layer8_ecg(ecg)
    ecg = tmp + ecg
    tmp = self.layer9_ecg(ecg)
    ecg = tmp + ecg
    ecg = self.maxpool_ecg_layer9(ecg)
    ecg = self.layer10_ecg(ecg)
    tmp = self.layer11_ecg(ecg)
    ecg = tmp + ecg
    ecg = self.maxpool_ecg_layer11(ecg)
    tmp = self.layer12_ecg(ecg)
    ecg = tmp + ecg
    ecg = torch.flatten(ecg, 1)
    ecg = F.relu(self.linear_ecg(ecg))

    art = self.encoder_art(art)
    tmp = self.layer1_art(art)
    art = tmp + art
    art = self.maxpool_art_layer1(art)
    tmp = self.layer2_art(art)
    art = tmp + art
    tmp = self.layer3_art(art)
    art = tmp + art
    art = self.maxpool_art_layer3(art)
    tmp = self.layer4_art(art)
    art = tmp + art
    tmp = self.layer5_art(art)
    art = tmp + art
    art = self.maxpool_art_layer5(art)
    art = self.layer6_art(art)
    tmp = self.layer7_art(art)
    art = tmp + art
    art = self.maxpool_art_layer7(art)
    tmp = self.layer8_art(art)
    art = tmp + art
    tmp = self.layer9_art(art)
    art = tmp + art
    art = self.maxpool_art_layer9(art)
    art = self.layer10_art(art)
    tmp = self.layer11_art(art)
    art = tmp + art
    art = self.maxpool_art_layer11(art)
    tmp = self.layer12_art(art)
    art = tmp + art
    art = torch.flatten(art, 1)
    art = F.relu(self.linear_art(art))

    eeg = self.encoder_eeg(eeg)
    tmp = self.layer1_eeg(eeg)
    eeg = tmp + eeg
    eeg = self.maxpool_eeg_layer1(eeg)
    tmp = self.layer2_eeg(eeg)
    eeg = tmp + eeg
    tmp = self.layer3_eeg(eeg)
    eeg = tmp + eeg
    eeg = self.maxpool_eeg_layer3(eeg)
    tmp = self.layer4_eeg(eeg)
    eeg = tmp + eeg
    tmp = self.layer5_eeg(eeg)
    eeg = tmp + eeg
    eeg = self.maxpool_eeg_layer5(eeg)
    eeg = self.layer6_eeg(eeg)
    tmp = self.layer7_eeg(eeg)
    eeg = tmp + eeg
    eeg = self.maxpool_eeg_layer7(eeg)
    tmp = self.layer8_eeg(eeg)
    eeg = tmp + eeg
    tmp = self.layer9_eeg(eeg)
    eeg = tmp + eeg
    eeg = self.maxpool_eeg_layer9(eeg)
    eeg = self.layer10_eeg(eeg)
    tmp = self.layer11_eeg(eeg)
    eeg = tmp + eeg
    eeg = self.maxpool_eeg_layer11(eeg)
    tmp = self.layer12_eeg(eeg)
    eeg = tmp + eeg
    eeg = torch.flatten(eeg, 1)
    eeg = F.relu(self.linear_eeg(eeg))

    combined = F.relu(self.linear_combined1(torch.cat((ecg, art, eeg), -1)))
    logits = self.linear_combined2(combined)
    # probs = F.sigmoid(logits)

    return logits

model = my_model()

## **Training**

For this draft, training will be limited to the 3 minute data

In [ ]:
def train_model_one_iter(train_dataloader, model, loss_func, optimizer):

  model.train()
  running_loss = 0

  for (ecg, art, eeg), y in train_dataloader:

    logits = model(ecg, art, eeg)
    loss = loss_func(logits.view(logits.shape[0]), y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  epoch_loss = running_loss / len(train_dataloader)

  return model, epoch_loss

In [ ]:
loss_func = nn.BCEWithLogitsLoss()
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

num_epoch = 10

# model training loop: it is better to print the training/validation losses during the training

# 3 min model
for i in range(num_epoch):

  model, train_epoch_loss = train_model_one_iter(train_loader_3min, model, loss_func, optimizer)
  print(f'Epoch: {i}, Train loss: {train_epoch_loss}')


In [ ]:
# Creat general checkpoint

# 3 min model
EPOCH = 9
PATH = "/content/drive/MyDrive/VitalDB/model_3min.pt"
LOSS = 0.37147167262057185

torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)

# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)


In [ ]:
# Load the general checkpoint

# 3 min model
model = my_model()
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

PATH = "/content/drive/MyDrive/VitalDB/model_3min.pt"

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

## **Evaluation**

In [ ]:
def eval_model(test_dataloader, model):

    model.eval()
    Y_score = []
    Y_pred = []
    Y_true = []

    for (ecg, art, eeg), y in test_dataloader:

        y_hat = torch.sigmoid(model(ecg, art, eeg))
        y_hat = y_hat.detach()
        Y_score.append(y_hat)

        y_hat = (y_hat>0.5).int()
        Y_pred.append(y_hat)

        Y_true.append(y)

    Y_score = np.concatenate(Y_score, axis=0)
    Y_pred = np.concatenate(Y_pred, axis=0)
    Y_true = np.concatenate(Y_true, axis=0)

    return Y_score, Y_pred, Y_true

In [ ]:
y_score, y_pred, y_true = eval_model(test_loader_3min, model)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

acc = accuracy_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_score)

print(f'Test Accuracy: {acc}')
print(f'Test AUC: {roc_auc}')

The accuracy is currently significantly lower than in the original paper, likely because i) I only used 2,000 patients and ii) there are many details the authors don't comment on (see comments above) which may have led to this model being different from the one they used

In [ ]:
# metrics to evaluate my model

# plot figures to better show the results

# it is better to save the numbers and figures for your presentation.

## Model comparison

In [ ]:
# compare you model with others
# you don't need to re-run all other experiments, instead, you can directly refer the metrics/numbers in the paper

# Discussion

Next steps:
- Include additional patients (currently, only 2,000 patients are included in this prelim analysis)
- Train models for the 5 min, 10 min, and 15 min data
- Compute AUPRC and optimal sensitivity / specificity for comparison with original paper
- Perform ablations:
  - Test the perfomance of ECG, blood pressure, and EEG data separately
  - test other optimizers
  - test different learning rates
  - test different batch sizes

# References

1.   Sun, J, [paper title], [journal title], [year], [volume]:[issue], doi: [doi link to paper]



# Feel free to add new sections